In [1]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

2024-08-27 14:40:10.036795: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-27 14:40:10.041036: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-27 14:40:10.084690: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-27 14:40:10.981486: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
x = torch.arange(-5, 5, 0.1).view(-1, 1)
y = -5 * x + 0.1 * torch.randn(x.size())

model = torch.nn.Linear(1, 1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

def train_model(iter):
    for epoch in range(iter):
        y1 = model(x)
        loss = criterion(y1, y)
        writer.add_scalar("Loss/train", loss, epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        for tag, param in model.named_parameters():
            writer.add_histogram(tag, param.grad.data, epoch)

train_model(10)
writer.flush()

In [10]:
writer.close()

In [11]:
import os, pwd
from tensorboard import notebook
import getpass
from IPython.core.display import display, HTML

def get_pid_owner(pid):    # the /proc/PID is owned by process creator    
    proc_stat_file = os.stat("/proc/%d" % pid)    
    # get UID via stat call    
    uid = proc_stat_file.st_uid    
    # look up the username from uid    
    username = pwd.getpwuid(uid)[0]        
    
    return username

def get_tb_port(username):        
    for tb_nb in notebook.manager.get_all():        
        if get_pid_owner(tb_nb.pid) == username:            
            return tb_nb.port    
        
def tb_address():        
    username = getpass.getuser()    
    tb_port = get_tb_port(username)        
    address = "https://jupyter.olcf.ornl.gov" + os.environ['JUPYTERHUB_SERVICE_PREFIX'] + 'proxy/' + str(tb_port) + "/"    
    address = address.strip()
    
    display(HTML('<a href="%s">%s</a>'%(address,address)))
    
# %load_ext tensorboard  
%reload_ext tensorboard

username = os.environ['JUPYTERHUB_USER']
# log_dir = os.path.expandvars('/lustre/orion/proj-shared/geo152/hackathon_Fall24')
log_dir = os.path.expandvars('runs')
print(log_dir)

runs


/tmp/ipykernel_520/3854095562.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [12]:
%%capture

UsageError: %%capture is a cell magic, but the cell body is empty.


In [13]:
%tensorboard --logdir $log_dir --port 0

tb_address()

Reusing TensorBoard on port 33327 (pid 550), started 0:02:52 ago. (Use '!kill 550' to kill it.)